In [1]:
import setup as s
import pandas as pd
import evaluate as e
import random as rand

In [2]:
evids =pd.read_json("data\\release_evidences.json")
evids1 = pd.read_json("data\\release_evidences.json")
conds  = pd.read_json('data\\release_conditions.json')
evids1 = evids1.T
print(evids1.columns)
# evids['numeric_part'] = evids['name'].apply(lambda x: int(x.split('_')[1]))
# sorted_evids = evids.sort_values(by='numeric_part')
# print(sorted_evids['name'].to_list())

Index(['name', 'code_question', 'question_fr', 'question_en', 'is_antecedent',
       'default_value', 'value_meaning', 'possible-values', 'data_type'],
      dtype='object')


In [3]:
conds = conds.T
conds.drop(['cond-name-fr', 'antecedents','cond-name-eng','icd10-id'], axis=1, inplace=True)
print(conds.columns)


Index(['condition_name', 'symptoms', 'severity'], dtype='object')


In [4]:
cond_evi = {}
int_to_cond ={}
for i in range(len(conds)):
   condition=  conds.iloc[i]
   name =condition['condition_name']
   cond_evi[name] = set(condition['symptoms'].keys())
   int_to_cond[i]=name



In [5]:
import pickle as p
with open('trained_model.bin', 'rb') as f:
    rf =  p.load(f)
    cols  = p.load(f)
print(len(cols))

914


In [6]:
multi_evi = s.gen_multichoice_features(evids)[1]
ordinal_evi_sc= s.gen_singlechoice_features(evids)[1]
categoical_evi_sc = s.gen_singlechoice_features(evids)[2]
binary_evi =s.gen_binary_features(evids)
print(len(ordinal_evi_sc+  categoical_evi_sc+   multi_evi+      binary_evi),sep ='\n'*3)
all_evis = multi_evi+ordinal_evi_sc+categoical_evi_sc+binary_evi
print(rand.choice(all_evis))
# print(len(ordinal_evi_sc),len(categoical_evi_sc),len(multi_evi),len(binary_evi),sep ='\n'*3)



223
E_16


In [7]:
import thompsonsampling as ts


In [12]:
picked_bandit = ts.pick_bandit()
disease = int_to_cond[picked_bandit]
symp_set = cond_evi[disease]
enquiry_chosen = rand.choice(list(symp_set))
print(disease,enquiry_chosen,symp_set)
print(evids1.loc[enquiry_chosen,"question_en"])

print(enquiry_chosen in categoical_evi_sc,enquiry_chosen in binary_evi,enquiry_chosen in multi_evi,enquiry_chosen in ordinal_evi_sc)
if 

Bronchiectasis E_201 {'E_66', 'E_45', 'E_77', 'E_201'}
Do you have a cough?
False True False False
